# Audio-to-audio generation 


In [1]:
import gin

gin.enter_interactive_mode()

from IPython.display import display, Audio
import torch
import numpy as np
import librosa
import sys

sys.path.append('..')
torch.set_grad_enabled(False)

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### Checkpoint setup

In [12]:
model_path = ""
step = 0
autoencoder_path = ""
device = "cpu"

### Instantiate te model and load the checkpoint

In [ ]:
from after.diffusion import RectifiedFlow

checkpoint_path = model_path + "/checkpoint" + str(step) + "_EMA.pt"
config = os.path.join(model_path, "config.gin")

# Parse config
gin.parse_config_file(config)
SR = gin.query_parameter("%SR")
n_signal = gin.query_parameter("%N_SIGNAL")

# Emb model

# Instantiate model
blender = RectifiedFlow(device=device)

# Load checkpoints
state_dict = torch.load(checkpoint_path, map_location="cpu")["model_state"]
blender.load_state_dict(state_dict, strict=False)

# Emb model
emb_model = torch.jit.load(autoencoder_path).eval()
blender.emb_model = emb_model

# Send to device
blender = blender.eval().to(device)

### Load the dataset

In [14]:
from after.dataset import SimpleDataset
from IPython.display import display, Audio

db_path = ""
dataset = SimpleDataset(path=db_path, keys=["z"])


In [ ]:
z1 = dataset[0]["z"][..., :n_signal]
z2 = dataset[1]["z"][..., :n_signal]

z1, z2 = torch.tensor(z1).to(device).unsqueeze(0), torch.tensor(z2).to(
    device).unsqueeze(0)

x1, x2 = blender.emb_model.decode(
    z1).cpu().squeeze(), blender.emb_model.decode(z2).cpu().squeeze()

display(Audio(x1, rate=SR))
display(Audio(x2, rate=SR))

#### Generation

In [16]:
nb_steps = 10  #Number of diffusion steps
guidance = 1.0  #Classifier free guidance strength

In [ ]:
# Compute structure representation
time_cond1, time_cond2 = blender.encoder_time(z1), blender.encoder_time(z2)

# Compute timbre representation
zsem1, zsem2 = blender.encoder(z1), blender.encoder(z2)

time_cond = time_cond1
zsem = zsem2

# Sample initial noise
x0 = torch.randn_like(z1)

print("Transfer")

xS = blender.sample(
    x0,
    time_cond=time_cond,
    cond=zsem,
    nb_steps=nb_steps,
)

audio_out = blender.emb_model.decode(xS).cpu().numpy().squeeze()
display(Audio(audio_out, rate=SR))